##### <font color='blue'>
- <font color='blue'> The motivation of this data-challenge project is to practice what we did in the Kernel Methods AMMI 2020 for machine learning. 
- <font color='blue'> The way we do the practice is to classify a data set about transcription factor. During this classification task we should predict whether a DNA sequence region is binding site to a specific transcription factor. 
- <font color='blue'> For that we have been provided 2000 data sample data labeled to learn our model which should be able to classifies as possible as 1000 sample unlabled.

## <font color = 'blue'>               DATA PREPROCESSING

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_Xtr = pd.read_csv('Xtr.csv') # Loading the training data 
df_Ytr = pd.read_csv('Ytr.csv') # Loading the training label data
df_Xte = pd.read_csv('Xte.csv') # Loading the testing data

In [3]:
df_Xtr.iloc[0][1] # The information in the sample data of index 0

'GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGAACCACCCAGGCATTGTGGGGCTGCCCTGCCACCTGCTGGCCGCTCCTGGTGGCAG'

#### <font color='blue'> Here we are processing to numerize the training sequence data by replacing the characters A, C, G and G  with A=(1, 0, 0, 0), C=(0, 1, 0, 0), G=(0, 0, 1, 0), T=(0, 0, 0, 1) repectivly.

In [4]:
df = []
for i in range(df_Xtr.shape[0]):
    l = []
    for j in range(len(df_Xtr.iloc[i][1])):
        char = df_Xtr.iloc[i][1][j]
        if char == 'A':
            l = l+[1, 0, 0, 0]
        elif char == 'C':
            l = l+[0, 1, 0, 0]
        elif char == 'G':
            l = l+[0, 0, 1, 0]
        else :
            l = l+[0, 0, 0, 1]
    df.append(l)

In [5]:
df_train = pd.DataFrame(df)

#### <font color='blue'> The training Label data.

In [6]:
y = df_Ytr['Bound'].values 

#### <font color='blue'> For the testing set.

In [7]:
df_t = []
for i in range(df_Xte.shape[0]):
    l = []
    for j in range(len(df_Xte.iloc[i][1])):
        char = df_Xte.iloc[i][1][j]
        if char == 'A':
            l = l+[1, 0, 0, 0]
        elif char == 'C':
            l = l+[0, 1, 0, 0]
        elif char == 'G':
            l = l+[0, 0, 1, 0]
        else :
            l = l+[0, 0, 0, 1]
    df_t.append(l)

In [8]:
df_test = pd.DataFrame(df_t)

In [9]:
df_test = df_test.values

#### <font color ='blue'> Here we convert the type data into float which is required down for the mathematical computation.

In [10]:
df_train = df_train.astype('float') 

#### <font color ='blue'> Here is where we do feature that is the selection of the k best for training data.

In [11]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [12]:
test = SelectKBest(score_func=chi2, k=100)
fit = test.fit(df_train, y)
np.set_printoptions(precision=3)
df_train = fit.transform(df_train)

In [13]:
df_test = fit.transform(df_test)

#### <font color ='blue'> Here we show the shape of the testing data that has 2000 rows and 404 columns.

In [14]:
df_train.shape

(2000, 100)

#### <font color ='blue'> Here we show the shape of the testing data that has 1000 rows and 404 columns.

In [15]:
df_test.shape

(1000, 100)

#### <font color ='blue'> The splitting of the training data into training data set and validation data set.

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.25)

#### <font color = 'blue'> keeping the y_train value before changing into a vectors of components (-1, 1)

In [17]:
y_train_keep = y_train

In [18]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

1500
1500
500
500


#### <font color = 'blue'> changing the y_train into a vectors of components (-1, 1)

In [19]:
y_new = []
for i in range(len(y_train)):
    if y_train[i] == 0:
        y_new.append(-1)
    else:
        y_new.append(1)

In [20]:
y_train[:10]

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1])

In [21]:
y_train = np.array(y_new)
y_train[:10]

array([-1,  1,  1, -1, -1, -1,  1,  1,  1,  1])

## - <font color = 'blue'> Logistic Regression from scratch


#### - <font color = 'blue'> This functio return w, b the primal solutions from the dual results.

In [22]:
def get_primal_from_dual(alpha, X, y, hard_margin=False, C=None, tol=1e-10):
    # w parameter in vectorized form
    w = ((y * alpha).T.dot(X)).flatten()
    # w = X.dot(y[:, np.newaxis]*alphas)
    #print(alpha)
    # sv = Support vectors!
    # Indices of points (support vectors) that lie exactly on the margin
    # Filter out points with alpha == 0
    sv = (alpha > tol)
    # If soft margin, also filter out points with alpha == C
    if not hard_margin:
        if C is None:
            raise ValueError('C must be defined in soft margin mode')
            print(C)
        sv = np.logical_and(sv, (C - alpha > tol))
    #print('y',sv[:100])
    b = y[sv] - X[sv].dot(w)
    b = b[0]
    
    #Display results
    # print('Alphas = {}'.format(alpha[sv]))
    # print('Number of support vectors = {}'.format(sv.sum()))
#     print('w = {}'.format(w))
#     print('b = {}'.format(b))
    
    return w, b

# w, b = get_primal_from_dual(alphas, X_train, y_train, hard_margin=True)
# # plot_points_with_margin(X_train, y_train, w, b)

In [23]:
import cvxopt
solver='cvxopt'
from cvxopt import matrix, solvers

In [24]:
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    print(A)
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    print(G.shape)
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    # print(A.shape)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)

    return np.array(solution['x']).flatten()
solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

## - <font color = 'blue'> Soft-margin svm learning step

#### - <font color = 'blue'> After trying the hard margin svm supposing the are linearly separable, I didn't get a good accuracy.
    
#### - <font color = 'blue'> For that thing I suppose that the soft-margin is better for this data set.
#### - <font color = 'blue'> The objective is to maximise that soft-margin which is the same to minimize the objective function of the primal and maximize the ojective function with respect to their constraints. The primal and dual objective function for soft margin are define below.

***
<font color = 'blue'>
We will use a quadratic program (QP) solver `cvxopt` to find our own solution to SVM
    

```
cvxopt.solvers.qp(P, q[, G, h[, A, b]])
```
    
solves the quadratic program

$$
\begin{aligned}
\min_x & \, \frac{1}{2}x^\top P x + q^\top x \\
\mathrm{s.t. } \, & Gx \leq h \\
& Ax = b
\end{aligned}
$$
<font color = 'blue'>
- $P, q$ define the objective
- $G, h$ are all the inequality constraints
- $A, b$ are all the equality constraints

**Find $P$, $q$, $G$, $h$, $A$ and $b$ for the hard margin SVM**
***

#### <font color='blue'> 
    
#### - <font color='blue'> As we know on convex optimization problem, it is sometime simpler to get the solution from the dual than the primal.
#### - <font color='blue'>That's we we define this function <font color='green'>'get_primal_from_dual' <font color='blue'> that we will call below. 

**Soft margin SVM, primal:**
$$
\begin{aligned}
\min_{w, b, \xi} & \, \frac{1}{2}w^\top w + C \mathbf{1}^\top \xi \\
\mathrm{s.t. } \, & \xi \geq 0 \\
& y_i x_i^\top w + y_i b + \xi_i\geq 1
\\
\end{aligned}
$$

- <font color='green'> $P = \frac{1}{2}I_p$ with 0 padding to make it $(p+1+n) \times (p+1+n)$
- $q = (0,..., 0, C, ..., C)$ </font> $q$ is $0$ $(p+1)$ times then $C$ $n$ times
$$$$
- <font color='green'> $G = -\left[X_y^\top, y, I_n\right]^\top, h = -\mathbf{1}_n^\top$</font>
- <font color='green'> $A = 0, b = 0$ </font>

**Soft margin SVM, dual:**
$$
\begin{aligned}
\max_\alpha & \, \mathrm{1}^\top\alpha -\frac{1}{2}\alpha^\top X_y^T X_y \alpha \\
\mathrm{s.t. } \, & \alpha \geq 0 \\
& \alpha \leq C \\
& y^\top\alpha = 0
\\
\end{aligned}
$$

- <font color='green'> $P = -X_y^T X_y$, $q = -\mathbf{1}$ </font>
- <font color='green'> $G = [-I, I]^\top, h = (0, ..., 0, C, ..., C)^\top$</font> $h$ is $0$ $n$ times then $C$ $n$ times
$$$$
- <font color='green'> $A = y^\top, b = 0$ </font>

#### <font color = 'blue'>
#### - <font color = 'blue'> This function the matrices P, G, A and the vectors q, h of the primal. 
#### - <font color = 'blue'> And finally we apply the solve_qp to return coefficient on which we use to primal optimal solution.

In [25]:
def svm_primal_soft_to_qp(X, y, C):
    n, p = X.shape
    assert (len(y) == n)
    
    Xy = np.diag(y).dot(X)
    # Primal formulation, soft margin
    diag_P = np.hstack([np.ones(p), np.zeros(n + 1)])
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    diag_P += eps
    P = np.diag(diag_P)
    
    q = np.hstack([np.zeros(p + 1), C * np.ones(n)])
    # y(wx+b)+ei>=1
    G1 = - np.hstack([Xy, y[:, np.newaxis], np.eye(n)])
    # ei>=0
    G2 = - np.hstack([np.zeros((n, p+1)), np.eye(n)])
    G = np.vstack([G1, G2])
    h = - np.hstack([np.ones(n), np.zeros(n)])
    A = None
    b = None
    return P, q, G, h, A, b
C = 10
coefs = solve_qp(*svm_primal_soft_to_qp(X_train, y_train, C=C))
n, p = X_train.shape
w_soft, b_soft, e = coefs[:p], coefs[p], coefs[(p+1):]


     pcost       dcost       gap    pres   dres
 0: -8.8934e+04  5.7797e+04  2e+05  2e+01  4e+00
 1:  2.6513e+04  6.0685e+03  2e+04  2e-15  6e-15
 2:  1.2543e+04  7.2743e+03  5e+03  1e-15  3e-14
 3:  1.1301e+04  8.1306e+03  3e+03  8e-16  3e-14
 4:  1.0409e+04  8.6684e+03  2e+03  5e-16  2e-14
 5:  9.8895e+03  9.0077e+03  9e+02  4e-16  2e-14
 6:  9.6190e+03  9.1865e+03  4e+02  4e-16  3e-14
 7:  9.4610e+03  9.2907e+03  2e+02  4e-16  8e-14
 8:  9.4009e+03  9.3326e+03  7e+01  4e-16  6e-14
 9:  9.3761e+03  9.3505e+03  3e+01  4e-16  1e-13
10:  9.3687e+03  9.3560e+03  1e+01  4e-16  5e-13
11:  9.3639e+03  9.3597e+03  4e+00  4e-16  3e-13
12:  9.3620e+03  9.3613e+03  7e-01  4e-16  8e-13
13:  9.3617e+03  9.3615e+03  1e-01  4e-16  3e-12
14:  9.3616e+03  9.3616e+03  2e-02  4e-16  1e-11
15:  9.3616e+03  9.3616e+03  7e-04  4e-16  2e-12
Optimal solution found.


#### <font color = 'blue'>
#### - <font color = 'blue'> This function the matrices P, G, A and the vectors q, h of the dual. 
#### - <font color = 'blue'> And finally we apply the solve_qp to return alpha on which we use to get the optimal solution of the dual.

In [26]:
def svm_dual_soft_to_qp(X, y, C):
    n, p = X.shape
    assert (len(y) == n)
    
    Xy = np.diag(y).dot(X)
    # Dual formulation, soft margin
    P = Xy.dot(Xy.T)
    # As a regularization, we add epsilon * identity to P
    eps = 1e-20
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    b = np.array([0.])
    A = A.astype('float')
    G = G.astype('float')
    return P, q, G, h, A, b
alphas = solve_qp(*svm_dual_soft_to_qp(X_train, y_train, C=C))
# print(alphas.shape)

     pcost       dcost       gap    pres   dres
 0: -6.0128e+03 -1.1285e+05  2e+05  5e-01  1e-12
 1: -6.0685e+03 -2.6513e+04  2e+04  1e-13  8e-13
 2: -7.2743e+03 -1.2543e+04  5e+03  4e-14  9e-13
 3: -8.1306e+03 -1.1301e+04  3e+03  6e-14  1e-12
 4: -8.6684e+03 -1.0409e+04  2e+03  4e-13  1e-12
 5: -9.0077e+03 -9.8895e+03  9e+02  9e-14  1e-12
 6: -9.1865e+03 -9.6190e+03  4e+02  2e-13  1e-12
 7: -9.2907e+03 -9.4610e+03  2e+02  1e-13  1e-12
 8: -9.3326e+03 -9.4009e+03  7e+01  4e-15  1e-12
 9: -9.3505e+03 -9.3761e+03  3e+01  2e-13  1e-12
10: -9.3560e+03 -9.3687e+03  1e+01  4e-14  1e-12
11: -9.3597e+03 -9.3639e+03  4e+00  3e-13  2e-12
12: -9.3613e+03 -9.3620e+03  7e-01  3e-14  1e-12
13: -9.3615e+03 -9.3617e+03  1e-01  3e-13  2e-12
14: -9.3616e+03 -9.3616e+03  2e-02  4e-15  1e-12
15: -9.3616e+03 -9.3616e+03  7e-04  2e-16  2e-12
Optimal solution found.


#### <font color = 'blue'>
#### - <font color = 'blue'> Goal: find the best possible $w$ and $b$
#### - <font color = 'blue'> Now we call the uncton <font color = 'green'>  'get_primal_from_dual' <font color = 'blue'> and apply it training data and the hyperparameters to find the weights corresponding to the optimal solutions. 
- <font color = 'blue'> Here the hyperparameters C is inversly propotional to the increasing of the overfitting.

In [27]:
w_soft, b_soft =  get_primal_from_dual(alphas, X_train, y_train, C=C ) 
# plot_points_with_margin(X_train, y_train, w, b)

#### <font color = 'blue'>
#### - <font color = 'blue'> SVM as a model:
#### - <font color = 'blue'> $\hat{y} = sign(w^\top x + b)$
$$$$
#### - <font color = 'blue'> Now we are making prediction <font color = 'green'> $\hat{y}$ <font color = 'blue'> which tell us weither a data point in the training data or testing data is labeled by $1$ or $-1$ that means it is bound or not bound.

In [28]:
def score_1(X, y, w, b):
    # print(X_train.shape)
    # print(w.shape)
    prediction = np.sign(X@w+b)
    # print(prediction[:4])
    y_new = [1 if pred == 1 else 0 for pred in prediction]

    sum_goodpred = 0

    for i in range(len(y)):
        if y[i]==y_new[i]:
            sum_goodpred+= 1
    accur = sum_goodpred/len(y_new)
    print('the accuracy for weight w, b is: {}%'.format((100*accur)))

#### <font color = 'blue'> Rechanging the y_train into vectors of components (0, 1)

In [29]:
y_train = y_train_keep 
y_train[:10]

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 1])

#### <font color = 'blue'> Checking the accuracy with respect to the yperparameter C.

In [30]:
score_1(X_train, y_train, w_soft, b_soft)

the accuracy for weight w, b is: 71.2%


In [31]:
score_1(X_test, y_test, w_soft, b_soft) 

the accuracy for weight w, b is: 67.80000000000001%
